In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install shap

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from imblearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
import shap
from datetime import datetime
import json
import joblib

In [ ]:
plt.rc('font', family='NanumBarunGothic')

In [ ]:
pd.options.display.float_format = '{:.6f}'.format

In [ ]:
# 파일 저장경로 및 sheet 설정
filename = '/content/drive/MyDrive/Colab Notebooks/전인CM/input/전인CM_통합데이터_221011.xlsx'
sheetname = '지산+의료+오피스빌딩'

In [ ]:
raw = pd.read_excel(filename, sheet_name=sheetname, engine='openpyxl')

In [ ]:
raw_data = raw.copy()

전처리

In [ ]:
nalist = []

for colname in raw_data.iloc[:, 0:17].columns : # 공종 비율 제외 결측 탐지 및 채우기 진행
  nacount = raw_data[colname].isna().sum()
  if nacount >= 1 :
    nalist.append(colname)

for col in nalist :  
  if raw_data[col].dtype == object :
    raw_data.loc[raw_data[col].isna()==True,col] = raw_data[col].mode()[0] # 최빈 처리
  else :
    raw_data.loc[raw_data[col].isna()==True,col] = raw_data[col].mean() # 평균처리

In [ ]:
# label encoding
raw_data['건물외형'] = raw_data['건물외형'].replace('정형', 0)
raw_data['건물외형'] = raw_data['건물외형'].replace('비정형', 1)
raw_data['철거공사 포함 여부'] = raw_data['철거공사 포함 여부'].replace('미포함', 0)
raw_data['철거공사 포함 여부'] = raw_data['철거공사 포함 여부'].replace('포함', 1)
raw_data['인테리어'] = raw_data['인테리어'].replace('유', 1)
raw_data['인테리어'] = raw_data['인테리어'].replace('무', 0)

sector = []
for i in range(len(raw_data)) :
  if raw_data['지역'][i] == '서울' : 
    sector.append(1)
  elif raw_data['지역'][i] == '인천' : 
    sector.append(2)
  elif raw_data['지역'][i] == '경기' : 
    sector.append(3)
  elif raw_data['지역'][i] == '충청' : 
    sector.append(4)
  elif raw_data['지역'][i] == '강원' : 
    sector.append(5)
  elif raw_data['지역'][i] == '영남' : 
    sector.append(6)
  elif raw_data['지역'][i] == '호남' : 
    sector.append(7)

raw_data['지역'] =  sector

In [ ]:
# 유사도 측정을 위한 공종 제외 feature 데이터 구축
encoding = pd.get_dummies(raw_data.drop(['공사비', '시공사명', '2015 평당가', '2015 기준공사비', '토목', '건축', '설비', '전기', '통신', '소방', '조경', '추가공종', '간접비 및 이윤'], axis=1), 
                       columns=['공법', '토목공법', '건물유형'], drop_first=True)

In [ ]:
# scaling을 위한 문자열 데이터 제거
column_list = encoding['프로젝트명']
column_data = pd.DataFrame(columns=column_list)
basic = encoding.drop(['프로젝트명'], axis=1)

In [ ]:
# scaling 작업
scaler = MinMaxScaler()
scaler.fit(basic)
scaled_data = scaler.transform(basic)
en_data = pd.DataFrame(data = scaled_data, index=basic.index, columns=basic.columns)
en_data = en_data.reset_index(drop=True)

In [ ]:
# 유사도 측정 로직 수정
for name in column_list : 
  data_list = []
  main_index = encoding[encoding['프로젝트명']==name].index
  main_values = en_data.values[main_index]
  main_year = en_data['착공년도'][main_index].values[0]

  for num in range(len(en_data)) : 
    compare_values = en_data.values[num]
    uclid_dist = np.sqrt(np.sum(np.square(main_values-compare_values)))
    if (main_year==en_data['착공년도'].min()) : 
      data_list.append(uclid_dist)
    else : 
      if (main_year < en_data['착공년도'][num]) :
        data_list.append(0)
      else : 
        data_list.append(uclid_dist)
  column_data[name] = data_list

In [ ]:
# 가장 유사도가 깊은 데이터 탐색
con_data = pd.concat([raw_data, column_data], axis=1)
sim = []
for name in raw_data['프로젝트명'] : 
  unique_data = column_data[name].unique()
  zero = [0]
  sim_data = np.setdiff1d(unique_data, zero).min()
  sim.append(sim_data)

In [ ]:
# 가장 유사도가 깊은 공사의 label 데이터 가져오기 
result = [] 
i = 0
for name in column_list : 
  sim_values = []
  result.append(con_data.loc[con_data[name]==sim[i]]['2015 기준공사비'].values[0])
  i += 1

In [ ]:
result_2 = []
i = 0
for name in column_list : 
  sim_values = []
  sim_values.append(con_data.loc[con_data[name]== sim[i]]['토목'].values[0])
  sim_values.append(con_data.loc[con_data[name]== sim[i]]['건축'].values[0])
  sim_values.append(con_data.loc[con_data[name]== sim[i]]['설비'].values[0])
  sim_values.append(con_data.loc[con_data[name]== sim[i]]['전기'].values[0])
  sim_values.append(con_data.loc[con_data[name]== sim[i]]['통신'].values[0])
  sim_values.append(con_data.loc[con_data[name]== sim[i]]['소방'].values[0])
  sim_values.append(con_data.loc[con_data[name]== sim[i]]['조경'].values[0])
  sim_values.append(con_data.loc[con_data[name]== sim[i]]['추가공종'].values[0])
  sim_values.append(con_data.loc[con_data[name]== sim[i]]['간접비 및 이윤'].values[0])
  result_2.append(sim_values)
  i += 1

In [ ]:
raw_data['유사도 기준공사비'] = result
raw_data.iloc[27, 17:26] = result_2[27]
raw_data.iloc[29, 17:26] = result_2[29]

모델링

In [ ]:
# baseline modeling을 위한 파라미터 설정
random_seed = 4
kfold = KFold(n_splits=5, shuffle=True, random_state=random_seed)

In [ ]:
drop_feature = ['프로젝트명', '공사비', '시공사명', '2015 평당가'] # 모든 데이터 사용
main_data = pd.get_dummies(raw_data.drop(drop_feature, axis=1), columns=['토목공법', '공법', '건물유형'], drop_first=True)
main_data = main_data[['착공년도', '연면적(평)', '지하층', '지상층', '층', '시공사 등급', '공사기간(개월)', '인테리어',
       '공사비지수', '2015 기준공사비', '토목', '건축', '설비', '전기', '통신', '소방', '조경', '추가공종',
       '간접비 및 이윤', '건물외형', '철거공사 포함 여부', '지역', '유사도 기준공사비', '공법_PRC ',
       '공법_RC', '공법_RC / PC', '공법_RC / 철골조', '공법_SRC', '공법_SRC / RC',
       '공법_SRC / SC', '토목공법_순타', '토목공법_역타', '건물유형_의료시설', '건물유형_지식산업센터']]

In [ ]:
idx = raw_data[list(map( lambda x : x in ['PC / PEB', 'PRC ', 'SRC / SC'], raw_data['공법']))].index

c = main_data.loc[idx,:]
A_c = c.drop(['2015 기준공사비'], axis=1)
b_c = c['2015 기준공사비']

d = main_data.loc[main_data.index.difference(idx),:]
A_d = d.drop(['2015 기준공사비'], axis=1)
b_d = d['2015 기준공사비']
A_train, A_test, b_train, b_test = train_test_split(A_d, b_d, test_size=0.3, random_state=random_seed) # 총 데이터 길이 27개 : 0.2 = 약 6개 / 3개 제외하고 사용하므로 0.3으로 설정하는 것으로

A_train = pd.concat([A_train, A_c], axis=0)
b_train = pd.concat([b_train, b_c], axis=0)

In [ ]:
# algorithm test 진행, 연관된 알고리즘을 리스트에 더 추가해도 무방
benchmark = pd.DataFrame(columns=['Algorithm', 'MAE', 'RMSE', 'MAPE', 'r2'])

for algorithm in [LinearRegression(), RandomForestRegressor(random_state=random_seed), XGBRegressor(random_state=random_seed),MLPRegressor(random_state=random_seed), Lasso(), Ridge(), GradientBoostingRegressor(random_state=random_seed)]:
    pipeline = Pipeline(steps = [['regressor', algorithm]])
    
    results = GridSearchCV(estimator=pipeline,
                           scoring= 'neg_mean_absolute_error',
                           param_grid ={},
                           cv = kfold,
                           n_jobs=-1)
    results.fit(A_train, b_train)
    results_pred = results.predict(A_test)
    mae = mean_absolute_error(b_test, results_pred)
    rmse = mean_squared_error(b_test, results_pred, squared=False)
    mape = mean_absolute_percentage_error(b_test, results_pred)
    r2_cof = r2_score(b_test, results_pred)
    benchmark = benchmark.append({'Algorithm' : algorithm, 'MAE' : mae,'RMSE' : rmse, 'MAPE' : mape, 'r2' : r2_cof}, ignore_index=True)

In [ ]:
benchmark

In [ ]:
opt_dict = {}
idx = benchmark['MAPE'].idxmin()
column_list = ['MAE', 'RMSE', 'MAPE', 'r2']
benchmark = benchmark.drop([idx], axis=0).reset_index(drop=True)

for columns in list(column_list) : 
  opt_dict['Average '+columns] = benchmark[columns].mean()

opt_dict  

In [ ]:
size = 0.2
X_train, X_test, y_train, y_test = train_test_split(A_d, b_d, test_size=size, random_state=7) # 10 8
X_train = pd.concat([X_train, A_c], axis=0)
y_train = pd.concat([y_train, b_c], axis=0)

pipeline_xgb = Pipeline(steps=[['regressor', XGBRegressor(objective='reg:squarederror', random_state=4)]])


params_xgb={'regressor__n_estimators' : [100, 200, 300, 500],
            'regressor__max_depth' : [3, 5, 8, 10], 
            'regressor__min_samples_split' : [2, 3, 5]}


grid_search_xgb = GridSearchCV(estimator=pipeline_xgb,
                               param_grid=params_xgb,
                               cv = kfold,
                               scoring = 'neg_mean_absolute_percentage_error',
                               n_jobs=-1)

grid_search_xgb.fit(X_train, y_train)
pred_xgb = grid_search_xgb.predict(X_test)
print(grid_search_xgb.best_params_)
print('MAE = {:.3f}'.format(mean_absolute_error(y_test, pred_xgb)))
print('RMSE = {:.3f}'.format(mean_squared_error(y_test, pred_xgb, squared=False)))
print('MAPE = {:.3f}'.format(mean_absolute_percentage_error(y_test, pred_xgb)))
print('r2 score = {:.3f}'.format(r2_score(y_test, pred_xgb)))

now = datetime.now()
f_mae = mean_absolute_error(y_test, pred_xgb)
f_rmse = mean_squared_error(y_test, pred_xgb, squared=False)
f_mape = mean_absolute_percentage_error(y_test, pred_xgb)
f_r2_cof = r2_score(y_test, pred_xgb)

metrics_dict = {}
metrics_dict['MAE'] = f_mae
metrics_dict['RMSE'] = f_rmse
metrics_dict['MAPE'] = f_mape
metrics_dict['r2_score'] = f_r2_cof

metrics_dict

In [ ]:
end_model = grid_search_xgb.best_estimator_['regressor']

In [ ]:
# SHAP feature importance 구축
explainer = shap.TreeExplainer(end_model, data=X_train) 
shap_values = explainer.shap_values(X_test) 
shap_obj = explainer(X_test)
shap.initjs()
shap.summary_plot(shap_values, X_test, plot_type = "bar")

In [ ]:
importances = np.absolute(shap_values).sum(axis=0) / shap_values.shape[0]
feature_importance = pd.Series(importances / np.sum(importances))
feature_importance.index = X_train.columns

In [ ]:
# 수정
numerical = raw_data.select_dtypes(include = 'number').columns.drop(['공사비', '2015 기준공사비', '2015 평당가'])
categorical = ['토목공법', '공법', '건물유형']
fe_list = list(feature_importance.index)

In [ ]:
cat_imp = []
for name in categorical : 
  sum = 0
  for i in range (len(fe_list)) : 
    if name in fe_list[i] :
      sum += feature_importance[i]
  cat_imp.append(sum)

In [ ]:
cat_fe = pd.Series(cat_imp)
cat_fe.index = categorical

In [ ]:
nu_fe = feature_importance[numerical]

In [ ]:
result_fe = pd.concat([nu_fe, cat_fe])
fe_dict = result_fe.to_dict()

In [ ]:
missing_rate = {}
feature_data = raw.drop(['공사비', '2015 기준공사비', '2015 평당가'], axis=1)

for name in feature_data.columns.tolist() : 
  cnt = 0
  for i in range(len(feature_data)) : 
    if feature_data[name].isnull()[i] == True : 
      cnt+=1
  missing_rate[name] = cnt/len(feature_data)

In [ ]:
main_dict = {}
main_dict['feature importance'] = fe_dict
main_dict['trainset_size'] = len(X_train)
main_dict['testset_size'] = len(X_test)
main_dict['total_size'] = len(X_train)+len(X_test)
main_dict['last_train_date'] = str(now)
main_dict['metrics'] = metrics_dict
main_dict['algorithm others'] = opt_dict
main_dict['feature missing rate'] = missing_rate

In [ ]:
main_dict

In [ ]:
json_file = json.dumps(main_dict)

In [ ]:
file_path = '/content/drive/My Drive/Colab Notebooks/전인CM/output/지산의료오피.json'

In [ ]:
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(json.dumps(main_dict, ensure_ascii=False))

In [ ]:
joblib.dump(end_model, '/content/drive/My Drive/Colab Notebooks/전인CM/output/지산의료오피_model.pkl')
joblib.dump(scaler, '/content/drive/My Drive/Colab Notebooks/전인CM/output/지산의료오피_scaler.pkl')
joblib.dump(con_data, '/content/drive/My Drive/Colab Notebooks/전인CM/output/지산의료오피_data.pkl')
joblib.dump(en_data, '/content/drive/My Drive/Colab Notebooks/전인CM/output/지산의료오피_en_data.pkl')
joblib.dump(explainer, '/content/drive/My Drive/Colab Notebooks/전인CM/output/지산의료오피_explainer.pkl')